In [1]:
import numpy as np
import ArtistExtractor as AE
import pandas as pd
import glob
import os
import LocationUtils as LU
from ExtractGenreClubs import SplitLineup

In [2]:
a = AE.createDictionnary()

In [3]:
files = []
files = glob.glob("ClubData/ClubData0/*.csv")+glob.glob("ClubData/ClubData1/*.csv")+glob.glob("ClubData/ClubData2/*.csv")+glob.glob("ClubData/ClubData3/*.csv")
print(len(files))
ClubDataFrame = pd.read_csv(files[0],sep="\t")

for file in files:
    df= pd.read_csv(file,sep ="\t")
    ClubDataFrame = ClubDataFrame.append(df,ignore_index = True)

#Drop Useless columns
ClubDataFrame = ClubDataFrame.drop('Unnamed: 0', 1)
ClubDataFrame.reset_index()
print(ClubDataFrame.shape)

1312
(10069, 7)


# Construct full artist dictionnary

In [4]:
files = []
files = glob.glob("Artists/*.txt")
print("Nb files : "+str(len(files)))
ArtistsMainDic = dict()
ArtistsAllDic = dict()

for file in files :
    filename = file.replace(".txt","")
    dictTemp = LU.loadDictionary(filename,path="")
    
    if("All" in file):
        ArtistsAllDic = {**ArtistsAllDic,**dictTemp}
        
    if("Main" in file):
        ArtistsMainDic = {**ArtistsMainDic,**dictTemp}
        
print("Artist dictionaries loaded. Merging...",end="")
LU.saveDictionary(ArtistsAllDic,"ArtistsAllGenres","Artists/",enc="UTF-8")
LU.saveDictionary(ArtistsMainDic,"ArtistsMainGenre","Artists/",enc="UTF-8")
print("done.")
print("Deleting temp files...")
for file in files :
    if("-" in file and "temp" in file):
        print("Deleting temp file "+file)
        os.remove(file)
print("DONE.")

Nb files : 9
Artist dictionaries loaded. Merging...done.
Deleting temp files...
DONE.


## Creating dataframe of artists

In [5]:
ArtistDicoMain = LU.loadDictionary("ArtistsMainGenreOld","Artists/",enc="UTF-8")
ArtistDicoAll =  LU.loadDictionary("ArtistsAllGenresOld","Artists/",enc="UTF-8")
ArtistsSet = set(a.encode("UTF-8") for a in ArtistDicoMain.keys())

ArtistsSet.update(ArtistDicoAll.keys())

DictionnaryGenre = LU.loadDictionary("MusicGenres","Artists/",enc="UTF-8")

ClubDataFrame["Genre"] = None
ClubDataFrame["All Genres"] = None
print("Retrieving genre over " +str(ClubDataFrame.shape)+" events:")

MissingGenres = set()

i=0
for id,row in ClubDataFrame.iterrows():
    genres = []
    lineup = row["LineUp"]
    artists = SplitLineup(lineup)
    
    for artist in artists:
        mainGenre = None
        allGenres = None
        
        if(artist in ArtistsSet):
            mainGenre = ArtistDicoMain.get(artist)
            if(mainGenre!="None" and mainGenre!=None):
                if(mainGenre not in DictionnaryGenre):
                    MissingGenres.add(mainGenre)
                else:
                    translatedGenre = DictionnaryGenre[mainGenre]
                if(translatedGenre!=None):
                    mainGenre = translatedGenre
                allGenres = ArtistDicoAll.get(artist)
        else:
            print("Not found in dictionnary :"+str(artist),end = "")
            #Updating dictionnaries
            ArtistsSet.add(artist)
            allGenres = AE.getGenre(artist,dictionnary=DictionnaryGenre,ReturnAllGenres=True)
            mainGenre = AE.getMaxGenre(allGenres,)
            print(", MainGenre is : "+str(mainGenre))
            ArtistDicoMain.update({artist : mainGenre})
            ArtistDicoAll.update({artist : allGenres})
        
        #Adding to LineUp genres
        genres.append(mainGenre) 
        
            
    if(len(genres)==0):
        print(artists)
    else:
        maxGenre =AE.getMaxGenre(genres)
        #updating dataframe
        ClubDataFrame = ClubDataFrame.set_value(id,"Genre",maxGenre)
        ClubDataFrame = ClubDataFrame.set_value(id,"All Genres",str(genres))
        ClubDataFrame = ClubDataFrame.set_value(id,"LineUp",str(artists))
    
    i+=1
    if(i%200==0):
        print(str(i))
    if(i%200==0):
        LU.saveDictionary(ArtistDicoMain,"ArtistDicoMain",path = "Artists/",enc="UTF-8")
        LU.saveDictionary(ArtistDicoAll,"ArtistDicoAll",path = "Artists/",enc="UTF-8")
        
print("FINISHED")
LU.saveDictionary(ArtistDicoMain,"ArtistDicoMain",path = "Artists/",enc="UTF-8")
LU.saveDictionary(ArtistDicoAll,"ArtistDicoAll",path = "Artists/",enc="UTF-8")

print("\n\n These genres were not found in dictionary : ")
print(MissingGenres)

Retrieving genre over (10069, 9) events:
Not found in dictionnary :Wolf Müller, MainGenre is : None
Not found in dictionnary :Kaan Düzarat, MainGenre is : None
Not found in dictionnary :Kombé, MainGenre is : None
Not found in dictionnary :Garçon, MainGenre is : None
Not found in dictionnary :Cepheì, MainGenre is : None
Not found in dictionnary :Nôze, MainGenre is : microhouse
Not found in dictionnary :Léonard de Léonard, MainGenre is : None
Not found in dictionnary :Andrés Marcos, MainGenre is : None
Not found in dictionnary :König Balthasar, MainGenre is : None
Not found in dictionnary :Christian & Raphaël, MainGenre is : r&b and soul
Not found in dictionnary :Stipé, MainGenre is : None
200
Not found in dictionnary :DJs Green Velvet  DJ Rush  Monika Kruse  Felix Kröcher  Moonbootica  RaudiveOliver Ho  DIM  Shonky  Xenia Beliayeva  Steve Land aka Randy  Hannah Holland  Dachshund  Agnès  Smilla  Phuture Traxx  TurntableBabes  Ulli & Maesoo  Banga  Beda  Sandro Kühne and others to be con

In [6]:
AE.createDictionnary()
ClubDataFrame[0:50]

,ClubID,ClubName,Address,EventID,EventName,Date,LineUp,Genre,All Genres
0,100121.0,Cyclo Café,"Boulevard de Pérolles 91, Fribourg",707619.0,"Light My Fridays: with Ariane Effect, DJ H-ERA...",10 April 2015,"['DJ HERA', 'Ariane Effect', 'Chris Ahn']",electronic,"['None', 'electronic', 'electronic']"
1,100121.0,Cyclo Café,"Boulevard de Pérolles 91, Fribourg",673119.0,Chris Ahn's 4AM Mixtape - Ep. 4,14 November 2014,['Chris Ahn'],electronic,['electronic']
2,100121.0,Cyclo Café,"Boulevard de Pérolles 91, Fribourg",672938.0,Chris Ahn's 4AM Mixtape - Episode 1,16 May 2014,['Chris Ahn'],electronic,['electronic']
3,100121.0,Cyclo Café,"Boulevard de Pérolles 91, Fribourg",707619.0,"Light My Fridays: with Ariane Effect, DJ H-ERA...",10 April 2015,"['DJ HERA', 'Ariane Effect', 'Chris Ahn']",electronic,"['None', 'electronic', 'electronic']"
4,100121.0,Cyclo Café,"Boulevard de Pérolles 91, Fribourg",673119.0,Chris Ahn's 4AM Mixtape - Ep. 4,14 November 2014,['Chris Ahn'],electronic,['electronic']
5,100121.0,Cyclo Café,"Boulevard de Pérolles 91, Fribourg",672938.0,Chris Ahn's 4AM Mixtape - Episode 1,16 May 2014,['Chris Ahn'],electronic,['electronic']
6,10186.0,Rocking Chair,"Avenue de Gilamont 60; 1800, Vevey",365245.0,Hacienda 30th Birthday Tour,05 May 2012,"['New Order', 'Underworld', 'Peter Hook']",rock,"['rock', 'rock', 'None']"
7,10186.0,Rocking Chair,"Avenue de Gilamont 60; 1800, Vevey",284885.0,Black & White Party,30 June 2010,['Lemonick'],r&b and soul,['r&b and soul']
8,10186.0,Rocking Chair,"Avenue de Gilamont 60; 1800, Vevey",49151.0,Jamie Lidell,23 April 2008,['Jamie Lidell'],r&b and soul,['r&b and soul']
9,102059.0,Acapella Bar,"Place de la Gare, 1957 Ardon, Valais, Suisse",693409.0,The Future Color Of Sound,17 April 2015,"['Dj Never Die', 'Jack Demon', 'Vitronic']",None,"['electronic', 'None', 'None']"


In [7]:
ClubDataFrame.to_csv("ClubData/EventsWithGenre.csv",encoding="UTF-8")